<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# IA Framework.
## Lab 1  - Introduction to Pyspark.
#### Part 1 Data munging with <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a>

**Resume**: The objective of this notebook is to discover [Spark](https://spark.apache.org/) framework and its python API [`PySpark`](http://spark.apache.org/docs/latest/api/python/). 
We will see the main motivation to use these frameworks that allow to apply distributed task on clusters and understand the concept of **RDD**(*Resilient Distributed Datasets*), main abstraction Spark provides , and how we can use them.

## Introduction

### When using Spark ?

When data becomes too big for either RAM or Disk memory and/or where computation time are too high for your computer.  
**Spark** allow you do parallelise your tasks to different clusters and provides an interface to do it easily. 

Machine learning algorithm (supervised or unsupervised) are iterative algorithm. Using them with *Hadoop* technology requires to read and write at each iteration on disk which will make learning really slow. [Spark](http://spark.apache.org/)' (*Resilient Distributed Dataset* or **RDD** [Zaharia et al. 2012](https://www.usenix.org/system/files/conference/nsdi12/nsdi12-final138.pdf). We won't get deeply into Hadoop details. See [Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099v2/document) for an introduction to *MapReduce*, its limitation. 

Spark allow to stock data in each cluster trough **RDD** and use them only in read-only mode which allow faster run of different algorithms.


#### About spark

* **Spark** can be used trough *Java*, *Scala*, *R* and *Python* API. We will see  [`PySpark`](http://spark.apache.org/docs/latest/api/python/) API all along this TP. 
* **Spark** has four main librairies:
 
    * [`SparlSQL`](https://spark.apache.org/docs/latest/sql-programming-guide.html) which allows to access really big and various kind of data, structured or not by executing SQL syntax. (Part 3).
    * [`MLlib`](http://spark.apache.org/docs/latest/ml-guide.html) which contains statistical and machine learning algorithm (Part 2 and 4).
    * [`SparkStreaming`](https://spark.apache.org/docs/latest/streaming-programming-guide.html) for live data stream processing (No cover in this TP).
    * [`GraphX`](https://spark.apache.org/docs/latest/graphx-programming-guide.html) for graphs and graph-parallel computation (No cover in this TP).



#### Spark locally
**Spark** is designed to run on cluster to take advantage of it.
In this TP we will run Spark locally to understand how to use *Pyspark* API, but we won't be able to realise the all potential of Spark.


#### Warnings

For the training phase of a machine learning model estimation, it's often easier to use bigger ressources (more Ram and CPU) than using **Spark** on distributed cluster. Using **Spark** (or **Hadoop**) will be more efficient for extracting sampling, preprocessing the data (See *Cdiscount* application in [Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099v2/document). 


#### References
Official [documentation](https://spark.apache.org/docs/latest/). [Karau et al. (2015)](http://index-of.co.uk/Big-Data-Technologies/Learning%20Spark%20%20Lightning-Fast%20Big%20Data%20Analysis%20.pdf).


## Configuration

### Configuration

**Spark** needs a configuration to be used. We create for that a `SparkConf` object that contains information about your application (where is the master, the cluster, etc.) and  `SparkContext` object which tell us how to access a cluster.

Here, these objects have already been defined within the pyspark kernel of your notebook, and can be directly used:

In [ ]:
sc

## Dataset
This notebook has been inspired by the one build by [J. A. Dianes](https://github.com/jadianes/spark-py-notebooks) which use the dataset used in the  [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) competition. It contains 9M of interactions within a network (see description [here](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names)). The objective of this contest was to detect attack in a network from various features build or computed on transaction or interaction with this network.

In [ ]:
import urllib.request
# Download the file from URL
DATA_PATH=""  #<-- Put here where you want to stock the file.
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",DATA_PATH+"kddcup.data_10_percent.gz")

## RDD

### Create a RDD

#### From  a python object
You can convert a python object to a RDD with the `parallelize` function.

In [ ]:
l = range(100)
l

In [ ]:
data = sc.parallelize(l)
data

#### From a file
The other way is to read a file from your computer as a RDD with the `textfile` function. We will now read the KDD dataset previously described.

In [ ]:
data_file = DATA_PATH+"kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
raw_data.take(2)

### **Transformations** and **Actions** on RDD.

They are two types of operations you can apply on RDD **Transformations** and **Actions**. 

* A **Transformation** allows to create a new RDD from an existing one.
* An **Action** applies computation on the RDD and return a value (like the `.count` function)

#### Map

The `.map` function is one of the simplest **transformation**. It allows to apply the same function to each entries of the RDD.
Below we apply a function to split each string entry of the RDD to get a list of elements :

In [ ]:
import time
ts = time.time()
csv_data = raw_data.map(lambda x : x.split(","))
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))

We can now observe the new RDD produced by the transformation : 

#### Take
The `.take` function is one of the simplest **action**. It convert the RDD to a python list of n element(s).

In [ ]:
import time
ts = time.time()
csv_take  =csv_data.take(2)
print(csv_take)
te =time.time()
print("Time execution :%.4f seconds" %(te-ts))

**Question** 
* What can you say about the execution’s time of the two cells above? Is it normal? Why?

#### A more complex map function

In the first example of the `.map` function in the cell above, the function applied on each entry has been first defined within the `lambda` operator. The `lambda` operator is a one-line statement function. It doesn’t allow you to define complex function.


You can also define first a python function and then apply it on the RDD. This will allow you to define more complex and more readable function.

Below is a function that convert element to float, if the string can be converted to a float, and separate features from the label for each line.

In [ ]:
# Fontion qui sépare les champ (elems=valeur) et extrait la 41ème = clef.
def parse_interaction(l):
    elems = l
    features =[]
    for e in  elems[:-1]:
        try:
            e=float(e)
        except ValueError:
            e=e
        features.append(e)
    y = elems[-1]
    return (features, y)
# Affichage des 5 premiers
key_csv_data = csv_data.map(parse_interaction)
print(key_csv_data.take(1))

#### Count

The `count` function is an **action** which allows to count number of elements that composes a RDD. 

Let's compute the number of element in the three RDDs created so far.

In [ ]:
ts = time.time()
print(raw_data.count())
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))


ts = time.time()
print(csv_data.count())
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))

ts = time.time()
print(key_csv_data.count())
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))

**Questions** 
* What can you say about the results of the three applications of the count function?
* What can you say about the execution’s times?

#### Distinct

The `distinct` function is a **transformation** that builds a RDD where all duplicated elements are removed.

In the cell below we build a list of all different protocols within the dataset:

In [ ]:
protocol = csv_data.map(lambda x : x[1]).distinct()
protocol.take(10)

#### Filter

Another very used **transformation** is the `filter` function. It creates a smaller dataset based on a custom condition function.

The cell below, will build a smaller RDD which contains only '.normal' entries.

In [ ]:
normal_csv_data = csv_data.filter(lambda x: 'normal.' == x[-1])

In [ ]:
t0 = time.time()
normal_count = normal_csv_data.count()
total_count = csv_data.count()
tt = time.time() - t0
print("They are %d normal interactions (over %d)" %(normal_count, total_count))

#### Collect
`collect` is an **action** similar to `.take` **action**,  except that it will convert the entire dataset to a python list. 

In [ ]:
t0 = time.time()
all_raw_data = raw_data.collect()
tt = time.time() - t0
print("Data collected in %.3f seconds" %tt)
all_raw_data[0]

In [ ]:
t0 = time.time()
print(len(all_raw_data))
tt = time.time() - t0
print("Time runing : %.3f" %tt)

#### Combining transformation

You've seen above that **transformations** are *lazy* operators. Combining various **transformations** on various RDD are also *lazy*, i.e. the all pipeline of functions will be run only when a **action** is applied.

In [ ]:
t0 = time.time()
data_file = DATA_PATH+"kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
key_csv_data = raw_data.map(parse_interaction)
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")
tt = time.time() - t0
print("Time runing : %.3f" %tt)

In [ ]:
t0 = time.time()
count_normal_interaction= normal_key_interactions.count()
tt = time.time() - t0
print("Time runing : %.3f" %tt)
print("There are %d 'normal' interactions" %count_normal_interaction)

## Operations on rdd.
Operations are **Transformation** that are applied on different RDDs. 

#### Subtract
The `Subtract` operation enables to create a RDD *C* from RDDs *A* and *B* by taking all entries from *A* that are not in *B*.

The cells below allow to create a RDD with no normal data in it.

In [ ]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x) #<- RDD with normal data
attack_raw_data = raw_data.subtract(normal_raw_data) #<-- RDD with 'anormal' data.

In [ ]:
t0 = time.time()
raw_data_count = raw_data.count()
tt = time.time() - t0
print("All count in {} secs".format(round(tt,3)))

In [ ]:
t0 = time.time()
normal_raw_data_count = normal_raw_data.count()
tt = time.time() - t0
print("Normal count in {} secs".format(round(tt,3)))

In [ ]:
t0 = time.time()
attack_raw_data_count = attack_raw_data.count()
tt = time.time() - t0
print("Attack count in {} secs".format(round(tt,3)))

In [ ]:
print("Il y a {} interactions normales et {} attaques, pour un total de {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count))

#### Cartesian

The cartesian product (`.cartesian`) returns every possible pairs between elements of two RDDs. (To be used with caution if RDDs are really big).

**Exercise**
Write function do display all possible pair of existing protocols (second columns) and services (third column) within the dataset using  `.cartesian` function.

In [ ]:
# %load solutions/exercise1_1.py

### More **Actions** on RDDs
So far we have used only *native* action function (count, distinct, take etc..) We will now define custom action with `reduce` and `aggregate` functions.

#### reduce

The `reduce` function takes a function as an argument that describes how elements from the RDD are combined.

The code below will build two RDDs containing duration time from normal and attack data.

In [ ]:
normal_csv_data =csv_data.filter(lambda x : x[-1]=="normal.")
attack_csv_data =csv_data.filter(lambda x : x[-1]!="normal.")

normal_duration_data = normal_csv_data.map(lambda x: int(x[0]))
attack_duration_data = attack_csv_data.map(lambda x: int(x[0]))

We use then the `reduce` function to compute the total duration of all these actions.

In [ ]:
total_normal_duration = normal_duration_data.reduce(lambda x, y: x + y)
total_attack_duration = attack_duration_data.reduce(lambda x, y: x + y)

print("Total duration for 'normal' interactions is {}".\
    format(total_normal_duration))
print("Total duration for 'attack' interactions is {}".\
    format(total_attack_duration))

We compute their means.

In [ ]:
normal_count = normal_duration_data.count()
attack_count = attack_duration_data.count()

print("Mean duration for 'normal' interactions is {}".\
    format(round(total_normal_duration/float(normal_count),3)))
print("Mean duration for 'attack' interactions is {}".\
    format(round(total_attack_duration/float(attack_count),3)))

The code above looks quite complicated for such a simple operation. The reduce function does not allow to define a "start" value, like in Python. It implies that you have to first convert the RDD in the wanted format. 
Here, you have to first convert the string information about the attack within integer before to count it. 


**Question** : How would you implement the `count` function with `reduce` function in Python?  Why those function wouldn’t work if it was applied with Spark?

In [ ]:
# %load solutions/exercise1_2.py


#### aggregate

The `aggregate` function allows to overcome the problem. The function takes as arguements : 
1. The initialization
2. A function that describes how elements of the rdd are combined
3. A function that describes how elements from different cluster will be combined

In [ ]:
normal_sum_duration = normal_duration_data.aggregate(0, # valeurs initiales à 0
    lambda acc, value: acc + value, # Somme des durées et cumul des interactions
    lambda acc1, acc2: acc1+ acc2 # cumul des accumualteurs
    )
normal_sum_duration

The aggregate function could be used to compute both total sum of duration and count of element in one call of the function.

**Exercise**: write a function that return total duraction of normal attack AND the count of normal attack

In [ ]:
attack_sum_count = attack_duration_data.aggregate(
    (0,0), # the initial value
    #todo;
    #todo
    )

print("Durée moyenne des interactions agressives {}".\
    format(round(attack_sum_count[0]/float(attack_sum_count[1]),3)))

In [ ]:
#% load solutions/exercise1_3.py

### ReducebyKey & CombineByKey

`ReducebyKey` & `CombineByKey` are functions that allow to apply reduce or aggregate function by key (which is the base of MapReduce function).

We need for that, to define a RDD where for each entry, the first element is the key, and the second element is the value.

In [ ]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0]))) # x[41] contient le type normal ou non
key_value_duration.take(1)

#### Reduce By Key


The `reduceByKey` function is used to apply reduce function for each key on the first columns.


In [ ]:
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)
durations_by_key.collect()

We note that `reduceByKey` is a **Transformation**

In [ ]:
counts_by_key = key_value_data.countByKey()
counts_by_key

#### combineByKey

`combineByKey` if for `aggregate` whate `reduceByKey` is to `reduce` function.


In [ ]:
sum_counts = key_value_duration.combineByKey(
    (lambda x: (x, 1)), # valeur initiale x and compteur 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # Combiner une paire  avec une paires d'accumulateurs (somme et incrément)
    (lambda acc1, acc2: (acc1[0]+acxc2[0], acc1[1]+acc2[1])) # combinaison des accumulateurs
     )
sum_counts.collect()

In [ ]:
duration_means_by_type = sum_counts.map(lambda lambda_args: (lambda_args[0], round(lambda_args[1][0]/lambda_args[1][1],3)))
duration_means_by_type.collect()